In [1]:
from fastai.text import *

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

In [3]:
df = pd.read_csv(path/'texts.csv', header=None)
df.head()

,0,1,2
0,label,text,is_valid
1,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
2,positive,This is a extremely well-made film. The acting...,False
3,negative,Every once in a long while a movie will come a...,False
4,positive,Name just says it all. I watched this movie wi...,False


In [4]:
data = TextLMDataBunch.from_csv(path, 'texts.csv')
# data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=42)

In [7]:
# from fastai_data import MusicTokenizer
# path = Path('data/midi/v6/midi_transcribe/shortdur/')
# data = TextLMDataBunch.load(path, cache_name='tmp/hook', bs=4, bptt=20)
# data.valid_ds.x.processor[0] = TokenizeProcessor(tokenizer=MusicTokenizer())
# t = data.train_ds[0][0]

In [8]:
# data.train_dl.sampler

# data.train_dl.dl.dataset

In [13]:
tdl = data.train_dl.new()

In [77]:
data.train_dl.sampler

In [6]:
it = iter(data.train_dl)

In [7]:
n = next(it)
[data.vocab.textify(x) for x in n[0]]

['" xxunk " had n\'t entered the general xxunk yet , but everyone knew the xxunk . xxmaj bruce could be heard on the radio singing , " xxmaj foreman says these jobs are going , boy , and they ai n\'t coming back to your hometown . " xxmaj xxunk had just been xxunk out by xxmaj uncle xxmaj sam . xxmaj bumper xxunk could be seen saying "',
 'in some secret hospital , a melting xxunk mass who goes xxunk and causes a xxunk nurse to run through a fake glass door . xxmaj apparently , xxmaj west " gets stronger as he melts " , which makes about as much sense as anything in this xxunk xxunk , xxunk - xxunk xxunk . xxmaj then this dopey " xxmaj army xxmaj brass " , who looks kind',
 'plot that has potential to be excellent all thrown together to form a pile of garbage . \n\n xxmaj first off the whole movie consists of terrible dialogue and god awful special affects . xxmaj the acting was also nothing to be proud of , but xxmaj xxunk xxmaj xxunk ( i think i spelled that right . ) saved the movie

In [8]:
n2 = next(it)
[data.vocab.textify(x) for x in n2[0]]

['xxmaj buy xxmaj american -- the job you save may be your own . " \n\n " xxmaj gung xxmaj ho " does a better job of capturing the mood of the xxmaj american industrial xxunk than just about any other popular movie made during that period . xxmaj certainly the movie has its flaws -- some loose plot threads and mediocre acting jobs by everyone except xxmaj michael xxmaj',
 'of like xxmaj coleman xxmaj francis ( director of many bad xxunk ) tries to cover the info up , but xxunk man runs around killing everyone he sees because he is melting . xxmaj he attacks a bickering old couple because he is melting . xxmaj he makes one terrible actress scream and xxunk xxunk for about 10 minutes because he is melting . xxmaj he is melting because',
 "'s sake : xxup don't xxup see xxup this xxup movie ! xxbos xxmaj first of all , in defense of xxup joan xxup fontaine , it must be said that xxmaj ginger xxmaj rogers would have been terribly miscast as xxmaj xxunk , the young xxmaj british lady who has

In [14]:
it2 = iter(tdl)

In [15]:
n = next(it2)
[data.vocab.textify(x) for x in n[0]]

["no matter what script he 's following . xxmaj his character , xxmaj hunt xxmaj stevenson , is a likable , affable xxunk , a natural leader with a wise - ass xxunk . xxmaj but he has a fatal flaw common to many of us : he does n't want to disappoint anyone . xxmaj he 'll distract the crowd with inspirational xxunk , and even lie , rather",
 'sex , then killed by human xxunk . xxmaj there is plenty of gore , including the classic " something is wrong with her , oh no , it \'s half the person she used to be " and the xxunk murder caused by panic . xxmaj but you can also find funny stuff like intestines pulled through someone \'s ass and a guy running in the woods then finding',
 'patience from the viewer . xxmaj it must be said that some of the humor falls flat and the usual romantic misunderstandings that occur in any xxmaj fred xxmaj astaire film of this period are given conventional treatment . xxmaj only the musical interludes give the story the lift it needs . \n\n xxmaj some pleas

In [70]:
n[0].shape

torch.Size([4, 20])